<a href="https://colab.research.google.com/github/dAn-solution/competition/blob/main/Prob_kiva_027.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kiva／クラウドファンディングの資金調達額予測
- アンサンブルの実施 第3層
- Prob_kiva_021 からLightGBM
- Prob_kiva_022 からRidge回帰
- Prob_kiva_023 から重回帰分析
- Prob_kiva_024 からLasso回帰
- Prob_kiva_025 からランダムフォレスト
- Prob_kiva_025 からGBDT（勾配ブースティング木）

### Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'

Mounted at /content/drive
/content/drive/My Drive


### ライブラリのインストール、インポート

In [ ]:
# カレントディレクトリを変更
import os
os.chdir('/content/drive/My Drive/Probdata/kiva/')
print(os.getcwd())

/content/drive/My Drive/Probdata/kiva


In [ ]:
class Config():
    root_path = './'
    input_path = os.path.join(root_path, 'input')
    output_path = os.path.join(root_path, 'output')
    result_path = os.path.join(root_path, 'result')
    bert_model_name = 'bert-base-uncased'
    constant = 25
    seed = 42
    debug = False

In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

### データの読み込み

In [ ]:
# train_df = pd.read_csv(f'{Config.input_path}/train.csv')
train_df = pd.read_csv(f'{Config.result_path}/half.csv')
sample_df = pd.read_csv(f'{Config.input_path}/sample_submission.csv')

train_021_df = pd.read_csv(f'{Config.result_path}/train_021.csv')
test_021_df = pd.read_csv(f'{Config.result_path}/test_021.csv')
train_022_df = pd.read_csv(f'{Config.result_path}/train_022.csv')
test_022_df = pd.read_csv(f'{Config.result_path}/test_022.csv')
# train_023_df = pd.read_csv(f'{Config.result_path}/train_023.csv')
# test_023_df = pd.read_csv(f'{Config.result_path}/test_023.csv')
# train_024_df = pd.read_csv(f'{Config.result_path}/train_024.csv')
# test_024_df = pd.read_csv(f'{Config.result_path}/test_024.csv')
train_025_df = pd.read_csv(f'{Config.result_path}/train_025.csv')
test_025_df = pd.read_csv(f'{Config.result_path}/test_025.csv')
train_026_df = pd.read_csv(f'{Config.result_path}/train_026.csv')
test_026_df = pd.read_csv(f'{Config.result_path}/test_026.csv')
train_029_df = pd.read_csv(f'{Config.result_path}/train_029.csv')
test_029_df = pd.read_csv(f'{Config.result_path}/test_029.csv')

## 重回帰分析の実行

In [ ]:
target_column = "LOAN_AMOUNT"
y = train_df[target_column]

In [ ]:
# 合体
# X = pd.concat([train_021_df, train_022_df, train_023_df, train_024_df, train_025_df, train_026_df, train_029_df], axis=1)
# test_X = pd.concat([test_021_df, test_022_df, test_023_df, test_024_df, test_025_df, test_026_df, test_029_df], axis=1)
X = pd.concat([train_021_df, train_022_df, train_025_df, train_026_df, train_029_df], axis=1)
test_X = pd.concat([test_021_df, test_022_df, test_025_df, test_026_df, test_029_df], axis=1)

In [ ]:
# test_021_df.max()
# test_X.columns = ['a','b','c','d','e','f']
print(X.describe())
test_X.describe()

                  0             0             0             0             0
count  45670.000000  45670.000000  45670.000000  45670.000000  4.567000e+04
mean     737.728408    739.434720    735.894635    737.243838  2.175689e+03
std      967.571806    958.396341    964.485065    954.847209  9.462118e+04
min       56.371777  -1031.387344     26.500000     14.824884 -1.135517e+04
25%      242.333850    239.160576    240.000000    244.983787  3.011579e+02
50%      463.393409    471.801978    462.500000    465.885616  4.557454e+02
75%      833.808081    824.338549    830.000000    835.585772  7.125310e+02
max     9663.027307   9786.562385  10000.000000   9900.910878  1.252437e+07


,0,0,0,0,0
count,91822.000000,91822.000000,91822.000000,91822.000000,9.182200e+04
mean,748.341663,769.353383,743.125346,745.385078,5.483720e+03
std,887.262294,867.863949,855.755859,860.162534,2.471460e+05
min,61.812490,-877.003945,50.000000,47.250189,-3.225455e+04
25%,345.202773,369.838657,341.500000,342.649282,3.757080e+02
50%,483.229406,499.541362,479.000000,473.751191,4.689872e+02
75%,789.650426,781.965918,786.500000,793.319136,6.621725e+02
max,9777.653976,8843.234368,9713.000000,9484.472750,4.287593e+07


In [ ]:
# trainデータを訓練用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = Config.seed)

In [ ]:
# 学習/予測

y_oof = np.zeros(len(y_train))
y_test_preds = []
y_preds = []
kf = StratifiedKFold(n_splits=5)
for fold, (tr_idx, vl_idx) in enumerate(kf.split(X_train, y_train)):
    x_tr_fold = X.iloc[tr_idx]
    y_tr_fold = y.iloc[tr_idx]
    x_vl_fold = X.iloc[vl_idx]
    y_vl_fold = y.iloc[vl_idx]

    model = LinearRegression()
    model.fit(x_tr_fold, y_tr_fold)
    y_oof[vl_idx] = model.predict(x_vl_fold)
    y_test_preds.append(model.predict(X_test))
    y_preds.append(model.predict(test_X))
    print(
        f'fold {fold} score:', mean_absolute_error(y_oof[vl_idx], y_vl_fold)
    )

print('X_test score:', mean_absolute_error(np.mean(y_test_preds, axis=0), y_test))

fold 0 score: 42.65925512031308
fold 1 score: 42.45734746232933
fold 2 score: 42.48714914185803
fold 3 score: 42.28434831744286
fold 4 score: 42.06010765259717
X_test score: 103.72242774778174


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [ ]:
model.fit(X, y)
print(mean_absolute_error(model.predict(X), y))
# y_preds = model.predict(test_X)

104.01982382909856


### 提出データの作成

In [ ]:
result = np.mean(y_preds, axis=0)
# result = y_preds
for i in range(len(result)):
    a, b = divmod(result[i], Config.constant)
    if b > (Config.constant/2):
        b = 1
    else:
        b = 0
    result[i] = (a + b) * Config.constant

In [ ]:
result[result<=0] = Config.constant 

In [ ]:
sample_df["LOAN_AMOUNT"] = result
print(sample_df)

       LOAN_ID  LOAN_AMOUNT
0      2041445        325.0
1      1944435        375.0
2      2083354        425.0
3      1993565        500.0
4      2064272        500.0
...        ...          ...
91817  1993862        325.0
91818  2015070        250.0
91819  1950349        100.0
91820  1921580       1175.0
91821  1976733        325.0

[91822 rows x 2 columns]


In [ ]:
# submitファイルのファイル名に利用する作成時刻の取得

now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=1)))
date_time = "{0:%Y%m%d_%H%M}".format(now)

In [ ]:
# submitファイルの出力(Google Driveに出力）
# 形式：submit_yyyymmdd_hhmm.csv

sample_df.to_csv(os.path.join(Config.output_path, f"submit_{date_time}.csv"), index=False)


In [ ]:
sample_df.describe()

In [ ]:
date_time

'20220202_1200'